In [1]:
from BCBio import GFF
import pandas as pd
import gffutils
from Bio import SeqIO
from collections import Counter
import pickle
import numpy as np
from numpy import inf
from math import log
import os.path


In [2]:

#db = gffutils.create_db("Vibrio_cholerae.GFC_11.37.gff3", dbfn='VC.db', force=True, keep_order=True)
db = gffutils.FeatureDB('VC.db', keep_order=True)
for cds in db.features_of_type('CDS'):
    #print(list(cds.astuple()))
    if list(cds.astuple())[8]!='0':
        print(list(cds.astuple())[1], list(cds.astuple())[8])
        

In [3]:
def argument_parse():
	ap = argparse.ArgumentParser()
	ap.add_argument('sequence_file')
	ap.add_argument('HMM_file')
	args = vars(ap.parse_args())
	sequence_file_name = args['sequence_file']
	config_file_name = args['HMM_file']
	print('Sequence file name:', sequence_file_name)
	print('HMM file name:', config_file_name)
	return sequence_file_name, config_file_name

def read_config_anno(filename):
    if(os.path.isfile('VC.db')==False): 
        print("creating database")
        db = gffutils.create_db(filename, dbfn='VC.db', force=True, keep_order=True)
    else:
        print("reading database")
        db = gffutils.FeatureDB('VC.db', keep_order=True)

    dict_keys=[]
    for cds in db.features_of_type('CDS'):
        current_line=list(cds.astuple())
        if current_line[1] not in dict_keys:
            dict_keys.append(current_line[1])
    
    anno_dict={k: [] for k in dict_keys}
    for cds in db.features_of_type('CDS'):
        current_line=list(cds.astuple())
        if current_line[7]=='+':
            segment = [(int(current_line[4]), int(current_line[5])),int(current_line[8])]
            anno_dict[current_line[1]].append(segment)
    print(anno_dict)
    return anno_dict
def remove_overlap(anno_dict):
    for keys in anno_dict:
        if len(anno_dict[keys])>0:
        #print(list(anno_dict[keys]))
            new=[]
            for ind,segs in enumerate(list(anno_dict[keys])[:-1]):
                #print(segs,anno_dict[keys][ind])


                if set(segs[0])&set(anno_dict[keys][ind+1][0])==True:
                    print(segs[0],anno_dict[keys][ind+1][0])
                    new_seg=[(segs[0][0],anno_dict[keys][ind+1][0][1]),segs[1]]
                    new.append(new_seg)
                else:
                    new.append(segs)
            print(len(anno_dict[keys]))
            print(list(anno_dict[keys])[len(anno_dict[keys])-2][0])
            #print(anno_dict[keys][len(anno_dict[keys])-1][0])
            if set(anno_dict[keys][len(anno_dict[keys])-1][0])&set(anno_dict[keys][len(anno_dict[keys])-2][0])==False:
                new.append(segs[-1])
            print(new)
            anno_dict[keys]=new
    return anno_dict


def read_config_seq(fasta):
    seq_dict={}
    for seq_record in SeqIO.parse(fasta, "fasta"):
        seq_dict[str(seq_record.id)]=str(seq_record.seq)
    return seq_dict
def initilize_codon_dict():
    codon_dictionary = {
    'TCA' : 0, 'TCC' : 0,    'TCG' : 0,    'TCT' : 0,  'TTC' : 0, 'TTT' : 0,    'TTA' : 0,    'TTG' : 0,    # Leucine
    'TAC' : 0, 'TAT' : 0,    'TAA' : 0,    'TAG' : 0,   'TGC' : 0,'TGT' : 0,    'TGA' : 0,    'TGG' : 0,    # Tryptophan
    'CTA' : 0, 'CTC' : 0,    'CTG' : 0,'CTT' : 0,   'CCA' : 0,  'CCC' : 0, 'CCG' : 0, 'CCT' : 0,    # Proline
    'CAC' : 0, 'CAT' : 0,    'CAA' : 0,    'CAG' :0,'CGA' : 0, 'CGC' : 0,  'CGG' : 0,   'CGT' : 0, 'ATA' : 0,    # Isoleucine
    'ATC' : 0, 'ATT' : 0,    'ATG' : 0,    'ACA' : 0, 'ACC' : 0,    'ACG' : 0,    'ACT' : 0,'AAC' : 0,    # Asparagine
    'AAT' : 0,  'AAA' : 0,  'AAG' : 0,  'AGC' : 0,  'AGT' : 0, 'AGA' : 0, 'AGG' : 0,'GTA' : 0,    # Valine
    'GTC' : 0,    'GTG' : 0,'GTT' : 0, 'GCA' : 0,   'GCC' : 0,    'GCG' : 0,'GCT' :0, 'GAC' : 0,  'GAT' : 0,  'GAA' : 0,
    'GAG' : 0,  'GGA' : 0, 'GGC' : 0,    'GGG' : 0, 'GGT' : 0,    # Glycine
    }
    return codon_dictionary
def cal_prob(dictionary, nb):
    for key in dictionary:
        dictionary[key]=dictionary[key]/nb
    return dictionary
        
def write_config(seq_dict,anno_dict,output):
#calculate average intergenic and gene region length
    inter_length_sum = 0
    inter_number_sum = 0
    gene_length_sum = 0
    gene_number_sum = 0
    seq_length_sum=0
    codon_number=0
    inter_nuc_dict = {'A':0, 'C':0, 'G':0, 'T':0}
    gene_codon_dict = initilize_codon_dict()
    start_codon_dict=initilize_codon_dict()
    stop_codon_dict=initilize_codon_dict()
    
    for seq_index in anno_dict:
        inter_start_index = 0
        print(seq_index)
        seq_length_sum+=len(seq_dict[seq_index])
        for seg in anno_dict[seq_index]:
            print("seq is",seg,seq_dict[seq_index][seg[0][0]-1:seg[0][1]])
            for i in seq_dict[seq_index][inter_start_index:seg[0][0]-1]:
                inter_nuc_dict[i] += 1
            inter_start_index = seg[0][1]
            #count gene area
            codon_list=[]
            print("START",seg[1])
            for i in range(seg[0][0]-1, seg[0][1], 3):
                codon_list.append(seq_dict[seq_index][i:i+3])
            for codon in codon_list[1:-1]:
                gene_codon_dict[codon] += 1
                codon_number+=1
            print(codon_list)
            #count start codon
            start_codon=codon_list[0]
            print("start",start_codon)
            start_codon_dict[start_codon]+=1
            #count stop codon
            stop_codon=codon_list[-1]
            print("stop",stop_codon)
            stop_codon_dict[stop_codon]+=1
            #calculate length
            gene_length_sum += seg[0][1] - (seg[0][0])+1
            gene_number_sum += 1
            inter_number_sum += 1

        for i in seq_dict[seq_index][inter_start_index:]:
            inter_nuc_dict[i] += 1
        inter_number_sum += 1

    inter_length_sum = seq_length_sum - gene_length_sum

    inter_length_avg = inter_length_sum / inter_number_sum
    gene_length_avg = gene_length_sum / gene_number_sum


    config_file = open(output, "w")
    config_file.write(str(inter_length_avg) + '\n')
    config_file.write(str(gene_length_avg) + '\n')
    config_file.write(str(inter_nuc_dict) + '\n')
    config_file.write(str(gene_codon_dict) + '\n')
    config_file.write(str(start_codon_dict) + '\n')
    config_file.write(str(stop_codon_dict) + '\n')
    config_file.close()
    
    inter_nuc_dict=cal_prob(inter_nuc_dict, inter_length_sum)
    gene_codon_dict=cal_prob(gene_codon_dict,codon_number)
    #gene_codon_dict['ATG']=0
    #gene_codon_dict['GTG']=0
    #gene_codon_dict['TTG']=0
    start_codon_dict=cal_prob(start_codon_dict,gene_number_sum)
    stop_codon_dict=cal_prob(stop_codon_dict,gene_number_sum)

    pickle.dump(inter_nuc_dict,open("intergenic_freq.pickle","wb"))
    pickle.dump(gene_codon_dict,open("genic_freq.pickle","wb"))
    pickle.dump(start_codon_dict,open("start_freq.pickle","wb"))
    pickle.dump(stop_codon_dict,open("stop_freq.pickle","wb"))

In [171]:
sequences=read_config_seq()

write_config(sequences,annotation,"frequency")

TypeError: read_config_seq() missing 1 required positional argument: 'fasta'

In [172]:
annotation=read_config_anno("Vibrio_cholerae.GFC_11.37.gff3")
annotation_new=remove_overlap(annotation)
#print(annotation_new)
#print(annotation)
sequences=read_config_seq('Vibrio_cholerae.GFC_11.dna.nonchromosomal.fa')
write_config(sequences,annotation_new,"frequency")

reading database
{'DN38.contig00001': [[(654, 2138), 0], [(2360, 4096), 0], [(8478, 9080), 0], [(17647, 18597), 0], [(28795, 29145), 0], [(29198, 29656), 0], [(31869, 32501), 0], [(32501, 34396), 0], [(34538, 37354), 0], [(38983, 39663), 0], [(42690, 44636), 0], [(46031, 46183), 0], [(46331, 47152), 0], [(48620, 49648), 0], [(49774, 50979), 0], [(50982, 52079), 0], [(52102, 52857), 0], [(53102, 53773), 0], [(53887, 54216), 0], [(54803, 55996), 0], [(59620, 60093), 0], [(64316, 65455), 0], [(69769, 70875), 0], [(82400, 83371), 0], [(83622, 84119), 0], [(84129, 85658), 0], [(92205, 92687), 0], [(92684, 92881), 0], [(96411, 97115), 0], [(97141, 98463), 0], [(98676, 99449), 0], [(99547, 100467), 0], [(104096, 105727), 0], [(105839, 107062), 0], [(107183, 108061), 0], [(115575, 116951), 0], [(117274, 117888), 0], [(118187, 119440), 0], [(119533, 120894), 0], [(121060, 122247), 0], [(122247, 124205), 0], [(124212, 125582), 0], [(126410, 128041), 0], [(128188, 129588), 0], [(131027, 132217), 

seq is [(232202, 232885), 0] ATGAAAAAAAATCAAATAGTAACAACGGAAGATATCCTACTCATGCTGTGCCAATCGGTTTCTACGGTTCTCACGTCCGCAACGAATTCACCGATTCACTATTCCGCAATGGTACAAAAAATTAATAAAACCTCGCTGAAACCAGATTTCGGCTGTTTTGTGTTGTTTGATGGCGGTTTTACTGGCCTTGTTGTGATCAACTTTACTGCCAAAGCGGCGCTTGAGGTTTACACCAATTACATGCGTAACATGGGCATGCCAGAAGAAGAACTCGCGGTATTACACACCTCCGATGAAGTTGGCGACGTACTGGGTGAGCTGATGAACCAGCTTGTCGGTGACTTCACTAACAAGGTGCGTAAAGAGCTGCAAACCAACATTACCCAGAACCAACCTAAGATGCTTTCATTGAACAAACAGGTGCTTTTATCTGTAGATACCAACCTTGATCGTCCACAGGCGCGACGTGTGACATTTTCAACAGCCAATAACAACATCTTTTATCTTGAACTGGCGATGGACAAAACCGAGTTTATCCAACTCGAAGAGTTCGACGTTCAGGAAGATGAAAGCCCTGACGATATTCTGGCCGCAACACAGAAACAGCAAAAATCACATTCAGTTGCTAACCACTCCGAAGAAGACAGTGCGTCTGCCGATTTGTTGGATCAGCTCGGCATCTAA
START 0
['ATG', 'AAA', 'AAA', 'AAT', 'CAA', 'ATA', 'GTA', 'ACA', 'ACG', 'GAA', 'GAT', 'ATC', 'CTA', 'CTC', 'ATG', 'CTG', 'TGC', 'CAA', 'TCG', 'GTT', 'TCT', 'ACG', 'GTT', 'CTC', 'ACG', 'TCC', 'GCA', 'ACG', 'AAT', 'TCA', 'CCG', 'ATT', 'CAC', 'TAT', 'TCC', 'GCA', 'ATG', 'GTA', 'CAA', 'AAA

seq is [(184771, 185148), 0] ATGAAACTGTTTTTAATTCTCATGCTTTCTGTATCGGGTGGCATCGCATCAGCGGATCACATCCAATCATTTATTCTCGGTTTCGGTGTGGCTTCTTTAGCCGTTGGTAGCTGTTACTGGCTTGCGTTTCGTAGTACGCGTTTTCCACAACTCGCCGTTTTTCTGCTCCTGTGTGGTTTCTTTGCTAAGGTTGCTGTAACCGTCATGGGCGTGATGTGGGGAATTTCAAATGAACTGATGACCTCGCCGTTTGTCTTTGCGTTGTCTTATCTCTTCTTCTCGATTGTCGCGACTTATTTCTGGTTCCGCTATCGTGAGTTGGCAACAAAACTCAAAGACAAAATTATCGCAACACAGCACCAATCGCACTCTCATTAA
START 0
['ATG', 'AAA', 'CTG', 'TTT', 'TTA', 'ATT', 'CTC', 'ATG', 'CTT', 'TCT', 'GTA', 'TCG', 'GGT', 'GGC', 'ATC', 'GCA', 'TCA', 'GCG', 'GAT', 'CAC', 'ATC', 'CAA', 'TCA', 'TTT', 'ATT', 'CTC', 'GGT', 'TTC', 'GGT', 'GTG', 'GCT', 'TCT', 'TTA', 'GCC', 'GTT', 'GGT', 'AGC', 'TGT', 'TAC', 'TGG', 'CTT', 'GCG', 'TTT', 'CGT', 'AGT', 'ACG', 'CGT', 'TTT', 'CCA', 'CAA', 'CTC', 'GCC', 'GTT', 'TTT', 'CTG', 'CTC', 'CTG', 'TGT', 'GGT', 'TTC', 'TTT', 'GCT', 'AAG', 'GTT', 'GCT', 'GTA', 'ACC', 'GTC', 'ATG', 'GGC', 'GTG', 'ATG', 'TGG', 'GGA', 'ATT', 'TCA', 'AAT', 'GAA', 'CTG', 'ATG', 'ACC', 'TCG', 'CCG', 'T

seq is [(174529, 174990), 0] ATGAAGATGAAAATTAACCGCCATGCTTATTATGGTTTGGTGCATAAAGGGGTGAAAGCGCTGCTACTCGATCGCATGGGTTTCTTTGATGACGATGAGTACCGCAACTACTTAGGCATACAAACAGGCAAAACTAGCTGTAAGGATTTAACCGATGGCGCACTGATGCAGTTAGTCAACGAGTTAAAGCAGCAAGGTTATTTGGAAGACTCGCCTAGCTTTAAAAAACGTTTGGGCGGCAGCTCATCCAATCAGCCATCCAGTAAGCAATGGGCTAAACTGGCGATATTAGCAAAATCTATTGGCTGGCAAGGGTTAGATGACCCTGCGCTCGATAGCTTTGTAAAACGCACCATTAAGGTGGAGCGTGCTCGCTGGCTCACTCGTGACAATATTCGTCAGGTGATTTTAGGGCTTGAGCGTTGGATTGCGGGTAAGGAGGCGTCATGTCACGGCGAGTAA
START 0
['ATG', 'AAG', 'ATG', 'AAA', 'ATT', 'AAC', 'CGC', 'CAT', 'GCT', 'TAT', 'TAT', 'GGT', 'TTG', 'GTG', 'CAT', 'AAA', 'GGG', 'GTG', 'AAA', 'GCG', 'CTG', 'CTA', 'CTC', 'GAT', 'CGC', 'ATG', 'GGT', 'TTC', 'TTT', 'GAT', 'GAC', 'GAT', 'GAG', 'TAC', 'CGC', 'AAC', 'TAC', 'TTA', 'GGC', 'ATA', 'CAA', 'ACA', 'GGC', 'AAA', 'ACT', 'AGC', 'TGT', 'AAG', 'GAT', 'TTA', 'ACC', 'GAT', 'GGC', 'GCA', 'CTG', 'ATG', 'CAG', 'TTA', 'GTC', 'AAC', 'GAG', 'TTA', 'AAG', 'CAG', 'CAA', 'GGT', 'TAT', 'TTG', 'GAA', 'GAC', 'TCG', 'C

seq is [(37659, 38903), 0] ATGATTTCACTGTGTGCGCTTGGGTATTTGCTGATCATTTTGGCCATCATTATTCCGCGCGGCATACAGAGTGTTGAGCAGCAAAAACAAGAGCTGGAAGAGAAGCTTGCGCTCTCTTTGAGTAACTCGGCGGCCATTGCACTGTATGTGAATAATTACGACATTGCCTCCGAGGTGATGGATGCGCTGCTGCTCCATAAAGAGATCAATGCGGTTAAGTTAGCCAGCGTGGATGGCATTGTCTTTGAACGAACCCCCATGCCCACAACCTACAAAGAACAGAATTATTGGACTGATGCGAACCGCTACCGATTAGATTCTCCCGTAGATGGTAATTTGATTGGCTATCTGATGATTCATGAGGATCATCAGGTGGTTCGTCAACAAGCCATTAATCAGATTCTTGATCAGATAGCCGTGGTTTTGATCCAGTTTCTCGTGACTTTTATTGCGCTTATCTGGATCGTTCGCCGCTTGGTCGGTAAACCCCTGACGGATCTTTCACAGGCGTTATCCGAAGTGCGGCCAGATCATGATCGCAAAGTGGCGGTAGCCGCAGAAGATCACCACAACGAGATTGGTTTGGTGGCCAAAAGCATCAATGAATTTATCGACGCTTCCCATCAAGCTTTACTGCGTGAACGCGAGCTGCGTCAACAGATTGAACGTTGGGAAAGCTACTATCGCACCATTGCCGAACAAGACACGCTGACGGGACTAAAAAACCGTTTAGGTTGTGAGAAGTTCGTGCTCCGCAAACAGCGTGCTAGCACCACCATGGTGCTGCTATTGATTGATTTGGATGGTTTTAAGCAGGTGAATGATACCTTGGGGCATGCGGCTGGCGATGAGGTATTACGTGAAATCGCTAAGCGTTTTTATGCCTTGGCGCAAACGCACTTTTCTGATTTTGTGGTCGGACGGCTCGGTGGTGATGAGTTTGCTATTTACATTCCGCTGGATGAATTCGTTG

['ATG', 'GTT', 'GAT', 'TTG', 'ATT', 'TGG', 'TTA', 'ACC', 'CCT', 'CCT', 'GTG', 'ATC', 'GCA', 'GGA', 'GGG', 'TCG', 'GTT', 'TTT', 'GTT', 'GTT', 'TTG', 'TGC', 'ATC', 'ATG', 'CTG', 'GCA', 'TTG', 'TGG', 'CGT', 'CTT', 'AAA', 'CGT', 'GGT', 'CAG', 'CAT', 'CGT', 'CAA', 'AGT', 'GAA', 'ACC', 'TTG', 'CGC', 'CAA', 'CAA', 'ATT', 'CGC', 'AAT', 'CTG', 'GAT', 'CGT', 'GAG', 'CTG', 'CAG', 'AAA', 'TCC', 'AAC', 'AAA', 'CAA', 'CTG', 'CTT', 'GAA', 'GTG', 'CGT', 'TCG', 'GTG', 'ATG', 'GTC', 'GGG', 'CTC', 'GGA', 'CAA', 'AAA', 'GTC', 'AGT', 'GAG', 'CAA', 'CAA', 'GAT', 'ATC', 'ATT', 'CGC', 'CAT', 'CTC', 'AAT', 'GAG', 'CGT', 'TTA', 'CTT', 'GAG', 'CTG', 'GAA', 'AAT', 'ACT', 'GAC', 'GCG', 'GAT', 'GCG', 'CGT', 'TTG', 'TAC', 'ACT', 'CGT', 'GCC', 'AGT', 'AAG', 'ATG', 'GTG', 'CAA', 'CTT', 'GGT', 'GCC', 'GAT', 'CTC', 'AAT', 'GAA', 'CTG', 'ATC', 'CAA', 'GAG', 'TGC', 'GAG', 'CTG', 'CCC', 'AAA', 'GCC', 'GAA', 'GCG', 'GAA', 'TTG', 'ATG', 'ATG', 'TCT', 'TTG', 'CAA', 'AAT', 'AAA', 'CTG', 'GCG', 'GGT', 'AAG', 'GAA', 'TCG', 'ATT'

stop TAG
seq is [(63797, 64291), 0] ATGGAATTACAGCAGTGTTGGAACAGTTTTTTGCAGGCTGAACAGTTACTCAAGCAAGGACACTGGTCGCAAGCCCATTACTTGTTAGAAGATGTTCTACACTACATGCCGCAACATATTCACAGTGCCGCACATGATGAGCAGACGAAACCATGCCAGATGGCATGTTTGATCACTGGTCTGCGTGATGCCGCTATTGCGCAGTCTGAAATTTTCAACAACATCGGCCAGTATCAACGCGCGTTTGATGCCCTCAATCAAGCCTATGCTCTGTTGCAATTTCTATCGATTGAAGAAATTGACTTGATGCGCCGCCTTAATACCACCCTATGCCAAGTGAGCGAATATCTGCTGCGCCATATCAATGCCTTTTGCTACGCGCAGCGCAGTGCCGAATGGCAGTTGGAATATCAACAAGTGGAGAAAGCACACTTCTACTTTAACCAGTTAAGAAGCAGCATGGCTTTACCCACCGCATCTCCCGTGATGAATTGA
START 0
['ATG', 'GAA', 'TTA', 'CAG', 'CAG', 'TGT', 'TGG', 'AAC', 'AGT', 'TTT', 'TTG', 'CAG', 'GCT', 'GAA', 'CAG', 'TTA', 'CTC', 'AAG', 'CAA', 'GGA', 'CAC', 'TGG', 'TCG', 'CAA', 'GCC', 'CAT', 'TAC', 'TTG', 'TTA', 'GAA', 'GAT', 'GTT', 'CTA', 'CAC', 'TAC', 'ATG', 'CCG', 'CAA', 'CAT', 'ATT', 'CAC', 'AGT', 'GCC', 'GCA', 'CAT', 'GAT', 'GAG', 'CAG', 'ACG', 'AAA', 'CCA', 'TGC', 'CAG', 'ATG', 'GCA', 'TGT', 'TTG', 'ATC', 'ACT', 'GGT', 'CTG', 'CGT', 'GAT', 'GCC', 'GCT', 'ATT

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [168]:
#print(annotation_new)
print(len(annotation))

121


In [4]:
def read_length(file):
    file1 = open(file, 'r') 
    Lines = file1.readlines() 
    return float(Lines[0].strip()),float(Lines[1].strip())

In [5]:
def initialize(intergenic_length,genic_length):
    intergenic_freq=pickle.load(open("intergenic_freq.pickle","rb"))
    genic_freq=pickle.load(open("genic_freq.pickle","rb"))
    start_freq=pickle.load(open("start_freq.pickle","rb"))
    stop_freq=pickle.load(open("stop_freq.pickle","rb"))
    
    initial_prob = np.array([1,0,0,0])
    transition_prob = {(0,0): 1-1/intergenic_length, (0,1):1/intergenic_length, (0,2):0, (0,3):0,
                (1,0):0, (1,1):0, (1,2):1, (1,3):0,
                (2,0): 0, (2,1):0, (2,2):1-3/genic_length, (2,3):3/genic_length,
                (3,0): 1, (3,1):0, (3,2):0, (3,3):0}

    emission_prob = {0:intergenic_freq, 1:start_freq, 2:genic_freq, 3:stop_freq}

    return initial_prob, transition_prob, emission_prob


In [6]:
def initialize2(intergenic_length,genic_length):
    initial_prob = {'I':1, 'S':0, 'M':0, 'T':0}
    transition_prob = {('I','I'): 1-1/intergenic_length, ('I','S'):1/intergenic_length, ('I','M'):0, ('I','T'):0,('S','I'):0, ('S','S'):0, ('S','M'):1, ('S','T'):0,('M','I'): 0, ('M','S'):0, ('M','M'):1-3/genic_length, ('M','T'):3/genic_length,('T','I'): 1, ('T','S'):0, ('T','M'):0, ('T','T'):0}
    intergenic_freq=pickle.load(open("intergenic_freq.pickle","rb"))
    genic_freq=pickle.load(open("genic_freq.pickle","rb"))
    start_freq=pickle.load(open("start_freq.pickle","rb"))
    stop_freq=pickle.load(open("stop_freq.pickle","rb"))
    emission_prob = {'I':intergenic_freq, 'S':start_freq, 'M':genic_freq, 'T':stop_freq}

    return initial_prob, transition_prob, emission_prob
def viterbi(seq):
    take_codon = False
    codon_flag = 1
    current_prob = {'I':emission_prob['I'][seq[0]], 'S':-inf, 'M':-inf, 'T':-inf} 
    path_trellis = [{'I':'I','S':'I','M':'I','T':'I'}]

    for ch in range(1, len(seq)-2):
        current_path = {}
        last_prob = current_prob.copy()

        if take_codon == True:
            codon_flag += 1
            if codon_flag > 3:
                codon_flag = 1
        if take_codon == False or codon_flag==1:
            codon = seq[ch:ch+3]
            codon_flag = 1
            if emission_prob['S'][codon]>0:
                take_codon = True

        for curr_state in initial_prob:
            prob_list = {}
            if curr_state == 'I':
                for prev_state in initial_prob:
                    if transition_prob[(prev_state,'I')]>0 and emission_prob['I'][seq[ch]]>0:
                        prob_list[prev_state] = last_prob[prev_state] + log(transition_prob[(prev_state,'I')]) + log(emission_prob['I'][seq[ch]])
                    else:
                        prob_list[prev_state] = -inf
                current_prob['I'] = max(prob_list.values())
                current_path['I'] = max(prob_list, key = prob_list.get)

            if curr_state=='S' or curr_state=='M' or curr_state=='T':
                if take_codon == True:
                    if codon_flag==1:
                        for prev_state in initial_prob:
                            if transition_prob[(prev_state,curr_state)]>0 and emission_prob[curr_state][codon]>0:
                                prob_list[prev_state] = last_prob[prev_state] + log(transition_prob[(prev_state,curr_state)]) + log(emission_prob[curr_state][codon])
                            else:
                                prob_list[prev_state] = -inf

                        current_prob[curr_state] = max(prob_list.values())
                        current_path[curr_state] = max(prob_list, key = prob_list.get)

                    elif codon_flag==2 or codon_flag==3:
                        current_path[curr_state] = curr_state
                else:
                    current_prob[curr_state] = -inf
        if emission_prob['T'][codon]>0 and codon_flag==3:
            take_codon = False
        path_trellis.append(current_path)
    state_seq = ''
    curr_state = max(current_prob, key = current_prob.get)
    for i in range(len(path_trellis)):
        index = len(path_trellis) - i - 1
        state_seq = path_trellis[index][curr_state] + state_seq
        curr_state = path_trellis[index][curr_state]

    return state_seq

def write_to_gff(seq_name,state_seq):
    #result_file = open(output, 'w')
    start = False
    start_index = 0
    end_index = 0
    result_file.write('###\n')
    for i in range(len(state_seq)):
        if start==False and state_seq[i]=='1':
            start_index = i
            start = True
        elif start == True and state_seq[i]=='3':
            end_index = i + 2
            start = False
            result_file.write(seq_name+'\t ena\t CDS \t {}\t {}\t . \t +\t 0\t .\n'.format(start_index, end_index))
        if end_index==0:
            result_file.write(seq_name+'\t ena\t CDS \t .\t .\t . \t +\t 0\t .\n')
    

In [104]:
def viter(sequence,intergenEmission,startcodon,codon,endcodon,transition):
    #viterbi=[[float('-inf')]*len(sequence) for i in range(4)]
    viterbi=np.full((4,len(sequence)),-inf)
    viterbi[0][0]=intergenEmission[sequence[0]]
    traces = np.zeros((4, len(sequence)), dtype = np.int)
    for i in range(1,len(sequence)):
        if i<3: 
            
            viterbi[0][i]=max(viterbi[0][i-1]+log(transition[(0,0)])+log(intergenEmission[sequence[i]]),viterbi[3][i-1]+log(intergenEmission[sequence[i]]))
            if viterbi[0][i]==viterbi[0][i-1]+transition[(0,0)]+intergenEmission[sequence[i]]:
                traces[0][i]=0
            else:
                traces[0][i]=3
        else:
            
            viterbi[0][i]=max(viterbi[0][i-1]+log(transition[(0,0)])+log(intergenEmission[sequence[i]]),viterbi[3][i-1]+log(intergenEmission[sequence[i]]))
            if viterbi[0][i]==viterbi[0][i-1]+log(transition[(0,0)])+log(intergenEmission[sequence[i]]):
                traces[0][i]=0
            else:
                traces[0][i]=3
            
            viterbi[1][i]=viterbi[0][i-3]+log(transition[(0,1)])+log(startcodon[sequence[i-2:i+1]])
            traces[1][i]=0
            
            viterbi[2][i]=max(viterbi[2][i-3]+log(transition[(2,2)])+log(codon[sequence[i-2:i+1]]),viterbi[1][i-3]+log(codon[sequence[i-2:i+1]]))
            if viterbi[2][i]==viterbi[2][i-3]+log(transition[(2,2)])+log(codon[sequence[i-2:i+1]]):
                traces[2][i]=2
            else:
                traces[2][i]=1
           
            viterbi[3][i]=viterbi[2][i-3]+log(transition[(2,3)])+log(endcodon[sequence[i-2:i+1]])
            traces[3][i]=2
    best_seq = np.zeros(len(sequence), dtype = np.int)
    best_seq[len(sequence) - 1] = (viterbi[:,len(sequence) - 1]).argmax()
    for i in reversed(range(len(sequence)-1)):
        from_idx = traces[:,(i+1)][best_seq[i+1]]
        best_seq[i] = from_idx

    print("best",best_seq)
    
        
    return np.array(viterbi),traces,best_seq

In [105]:
def back_tracing(viterbi,sequence,intergenEmission,startcodon,codon,endcodon,transition):
    best_seq=np.zeros(len(sequence), dtype = np.int)
    a,b=viterbi.shape
    start=(viterbi[:,b-1]).argmax()
    print(start)
    best_seq[b - 1] = start
    current=start
    for i in reversed(range(len(sequence))):
        print("best",best_seq[i:],"current",current,current==0)
        print(len(sequence)-1,i)
        if current==0:
            
            print("nuc is",sequence[i])
            if viterbi[0][i]==viterbi[0][i-1]+log(transition[(0,0)])+log(intergenEmission[sequence[i]]):
                    best_seq[i]=0
                    current=0
                    print("00")
            else:
                print("03")
                best_seq[i]=3
                start_codon=sequence[i-2:i+1]
                print("here",start_codon)
                current=3
        
        #start
        if current==1:
            if best_seq[i+1]==1 and best_seq[i+2]==1 and best_seq[i+3]==1:
                best_seq[i]=0
                current=0
                
            elif best_seq[i+1]==1 and best_seq[i+2]==1:
                print("11")
                best_seq[i]=1
                current=1
            elif best_seq[i+1]==1:
                print("111")
                best_seq[i]=1
                current=1
            else:
                print("10")
                best_seq[i]=1
                current=1
        #genic
        if current==2:
            print("genic",sequence[i-2-3:i+1])
            if best_seq[i+1]==2 and best_seq[i+2]==2 and best_seq[i+3]==2:
                if viterbi[2][i]==viterbi[2][i-3]+log(transition[(2,2)])+log(codon[sequence[i-2:i+1]]):
                    best_seq[i]=2
                    current=2
                else:
                    best_seq[i]=1
                    current=1
            elif best_seq[i+1]==2 and best_seq[i+2]==2:
                print("here2")
                best_seq[i]=2
                current=2
            elif best_seq[i+1]==2:
                print("here22")
                best_seq[i]=2
                current=2
            else:
                print("here222")
                best_seq[i]=2
                current=2
            
        if current==3:
            #print("stop",sequence[i-2:i+1])
            if best_seq[i+1]==3 and best_seq[i+2]==3 and best_seq[i+3]==3:
                best_seq[i]=2
                current=2
            elif best_seq[i+1]==3 and best_seq[i+2]==3:
                print("here3")
                best_seq[i]=3
                current=3
            elif best_seq[i+1]==3:
                print("here33")
                best_seq[i]=3
                current=3
            else:
                print("stop",sequence[i-2:i+1])
                print("here333")
                best_seq[i]=3
                current=3
    return best_seq
        

In [15]:

def fill_viterbi_table(obs,initiation,emission_list,transition):
    state=['I','S','G','T']
    take_codon = False
    codon_flag = 1
    vtable=np.full((4,len(obs)),-inf)
    ##to avoid underflow, we work in log space 
    start_arr=np.full((len(state)),emission_list[0][ obs[0]])
    vtable[:, 0] = initiation * start_arr
    #print(vtable)
    traces = np.zeros((len(state), len(obs)), dtype = np.int)
    best_seq = np.zeros(len(obs), dtype = np.int)
    count=-2    
    for t in np.arange(1, len(obs)-2):
        if take_codon == True:
            codon_flag += 1
            if codon_flag > 3:
                codon_flag = 1
        if take_codon == False or codon_flag==1:
            codon = obs[t:t+3]
            codon_flag = 1
            if emission_list[1][codon]>0:
                take_codon = True
        for j in range(len(state)): 
            if j!=0:
                if take_codon==True:
                    if codon_flag==1:
                        scores=[]
                        for prev in range(len(state)):
                            if transition[(prev,j)]>0 and emission_list[j][codon]>0:
                                scores.append(vtable[prev][t-1]+log(transition[(prev,j)])+log(emission_list[j][codon]))
                            else:
                                scores.append(-inf)
                        vtable[j][t]=max(scores)
                        traces[j][t]=scores.index(max(scores))
                    elif codon_flag==2 or codon_flag==3:
                        traces[j][t] = j
                        vtable[j][t]=vtable[j][t-1]
                else:
                    vtable[j][t] = -inf
            if j==0:
                scores=[]
                for s in range(len(state)):
                    if transition[(s,0)]>0 and emission_list[0][obs[t]]>0:
                        scores.append(vtable[s][t-1]+log(transition[(s,j)])+log(emission_list[j][obs[t]]))
                    else:
                        scores.append( -inf)
                vtable[j][t]=max(scores)
                traces[j][t]=scores.index(max(scores))
        if emission_list[3][codon]>0 and codon_flag==3:
            take_codon = False
                            
                                
    print(traces)                
                
    ##back-tracking to find the best hidden sequence
    best_seq[len(obs) - 1] = (vtable[:,len(obs) - 1]).argmax()
    for i in reversed(range(len(obs)-1)):
        from_idx = traces[:,(i+1)][best_seq[i+1]]
        best_seq[i] = from_idx

    print("best",best_seq)
    
    return best_seq
        

In [25]:
intergenic_length,genic_length=read_length("frequency2")
print(intergenic_length,genic_length)
initial_prob, transition_prob, emission_prob=initialize2(intergenic_length,genic_length)
initial_prob, transition_prob, emission_prob=initialize(intergenic_length,genic_length)



1057.8286852589642 991.4515103338633


In [9]:
from Bio import SeqIO
SEQS=[]
SEQ_NAME=[]
for seq_record in SeqIO.parse("Vibrio_cholerae.GFC_11.dna.nonchromosomal.fa", "fasta"):
    print(seq_record.id)
    print(repr(seq_record.seq))
    print(len(seq_record))
    SEQS.append(str(seq_record.seq))
    SEQ_NAME.append(str(seq_record.id))

DN38.contig00001
Seq('ATCGACGATAGACGCGTTTTCCAGCTTAGTGGTGAAGAAGTTTTCTTGTTTACC...ACC')
342055
DN38.contig00002
Seq('CGAAGTGCAACTGGCGAGTCGGGTATCCAATCTGACTGCTGAGCAGATGGTCGA...CCT')
198088
DN38.contig00003
Seq('ACCCATGATGAGGGCTTTTTCTATTGAGCAGATGCGGTATTTAGAGAGTGACAA...ATT')
191661
DN38.contig00004
Seq('CCTTGGGCGATGGCAGCAAGCTGGAAACCAAGGTGGTGGATCTGGGCAACGGCC...GGT')
156212
DN38.contig00005
Seq('AGTAGAACATTGCCAGGCTTTAATTTACAAAACACGTTACTTATAACGTGTTTT...AAG')
152225
DN38.contig00006
Seq('TTAGCAGGGGAGCGCCTTCAGCCTCTCGGCCACCTCACCACTTTCATATTCCAG...GAG')
150532
DN38.contig00007
Seq('TACCGACCGATACCAAAGAGTATCGTCAAATGGAAGCGGCGATGAAGCTTCGTG...TTT')
128098
DN38.contig00008
Seq('CCCCGATATTCGGGGCGTTTTTTATTGCTATCTCTCCTCAAACGACTACGAGTT...GAC')
123490
DN38.contig00009
Seq('GACACCCTACTATGGGGTGTCAATTTTTTTACTCGATTAATAAATCAGAGACTC...GGG')
110855
DN38.contig00010
Seq('CGTTTTGCTTCGGCTTTCTTACCGAAGAGGCTGTGCATTCTAGCGAGATAATTG...GCT')
101418
DN38.contig00011
Seq('GCAGATATTCCCTTATTAAGTCACGAACATATTCACTCAAACGAACATTAAATG...C

In [106]:
result_file = open('result.gff3', 'w')
print('{} sequences in total'.format(len(SEQS)))
for i in range(2):
    print('searching {}-th sequence'.format(i))
    print(SEQS[i][len(SEQS[i])-10:len(SEQS[i])])
    #state_seq = fill_viterbi_table(SEQS[i],initial_prob, emission_prob,transition_prob)
    table,state_seq,best_seq=viter(SEQS[i],emission_prob[0],emission_prob[1],emission_prob[2],emission_prob[3],transition_prob)
    best_seq=back_tracing(table,SEQS[i],emission_prob[0],emission_prob[1],emission_prob[2],emission_prob[3],transition_prob)
    print(best_seq)
    #for x in best_seq:
        #if x!=0:
            #print(x)
    #state_seq = viterbi(SEQS[i])
    #write_to_gff(SEQ_NAME[i],state_seq)
result_file.write('###\n')
result_file.close()

151 sequences in total
searching 0-th sequence
GGTAGCCACC


ValueError: math domain error